In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
covid_data =pd.read_parquet("COVID19DataForVoila.parquet.gzip")
covid_panel = covid_data.copy()

In [2]:
state_dict = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ',
    'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 
    'Delaware': 'DE', 'District of Columbia': 'DC', 'Florida': 'FL', 
    'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL',
    'Indiana': 'IN', 'Iowa': 'IA','Kansas': 'KS', 'Kentucky': 'KY',
    'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
    'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
    'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
    'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI',
    'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX',
    'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA',
    'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}

In [3]:
state_df = covid_data.reset_index()[["cumulative_cases", 
                                         "cumulative_deaths", 
                                         "total_population", 
                                         "state", 
                                         "date"]].copy()

state_df = state_df.groupby(["state", "date"]).sum()
#state_df.loc["United States", :] = np.nan
US_df =state_df.groupby("date").sum()
US_df["state"] = "United States"
US_df = US_df.reset_index().set_index(["state", "date"])
state_df = state_df.append(US_df)

cumulative_cases  cumulative_deaths  \
state         date                                              
Alabama       2020-01-22                 0                  0   
              2020-01-23                 0                  0   
              2020-01-24                 0                  0   
              2020-01-25                 0                  0   
              2020-01-26                 0                  0   
              2020-01-27                 0                  0   
              2020-01-28                 0                  0   
              2020-01-29                 0                  0   
              2020-01-30                 0                  0   
              2020-01-31                 0                  0   
              2020-02-01                 0                  0   
              2020-02-02                 0                  0   
              2020-02-03                 0                  0   
              2020-02-04                 0                  0   
              2020-02-05                 0                  0   
              2020-02-06                 0                  0   
              2020-02-07                 0                  0   
              2020-02-08                 0                  0   
              2020-02-09                 0                  0   
              2020-02-10                 0                  0   
              2020-02-11                 0                  0   
              2020-02-12                 0                  0   
              2020-02-13                 0                  0   
              2020-02-14                 0                  0   
              2020-02-15                 0                  0   
              2020-02-16                 0                  0   
              2020-02-17                 0                  0   
              2020-02-18                 0                  0   
              2020-02-19                 0                  0   
              2020-02-20                 0                  0   
...                                    ...                ...   
United States 2020-11-21          11974062             253466   
              2020-11-22          12117847             254353   
              2020-11-23          12281041             255211   
              2020-11-24          12451881             257312   
              2020-11-25          12640408             259553   
              2020-11-26          12752580             260791   
              2020-11-27          12950573             262152   
              2020-11-28          13100798             263340   
              2020-11-29          13239729             264164   
              2020-11-30          13391457             265281   
              2020-12-01          13571616             267836   
              2020-12-02          13773288             270524   
              2020-12-03          13989237             273361   
              2020-12-04          14213141             275836   
              2020-12-05          14424298             278058   
              2020-12-06          14602393             279164   
              2020-12-07          14783649             280475   
              2020-12-08          14999039             282988   
              2020-12-09          15226660             286062   
              2020-12-10          15449744             288957   
              2020-12-11          15678046             292138   
              2020-12-12          15893052             294493   
              2020-12-13          16080768             295867   
              2020-12-14          16266504             297259   
              2020-12-15          16462083             300224   
              2020-12-16          16713397             303947   
              2020-12-17          16944264             307277   
              2020-12-18          17190366             310035   
              2020-12-19          17384244             312585

In [4]:
state_df["Cases per Million"] = state_df[
    "cumulative_cases"].div(state_df["total_population"]) * 10 ** 6
state_df["Deaths per Million"] = state_df[
    "cumulative_deaths"].div(state_df["total_population"])* 10 ** 6

state_df["Daily Cases per Million MA"] = state_df[
    "cumulative_cases"].groupby("state").diff(1).div(
    state_df["total_population"]).rolling(7).mean() * 10 ** 6

state_df["Daily Deaths per Million MA"] = state_df[
    "cumulative_deaths"].groupby("state").diff(1).div(
    state_df["total_population"]).rolling(7).mean() * 10 ** 6

state_df.fillna(0, inplace = True)
state_df.reset_index(inplace = True)
state_df.set_index(["date", "state"], inplace = True)
state_df_pivot = pd.pivot_table(state_df.reset_index(), 
               values=['Cases per Million', "Deaths per Million",
                       "Daily Cases per Million MA", "Daily Deaths per Million MA"], 
               index=['date'], columns=['state'])

In [5]:
covid_data = pd.pivot_table(covid_data.reset_index(), 
               values=['Cases per Million', "Deaths per Million",
                       "Daily Cases per Million MA", "Daily Deaths per Million MA"], 
               index=['fips_code'], columns=['date'])

In [6]:
def import_geo_data(filename, index_col = "Date", FIPS_name = "FIPS"):
    # import county level shapefile
    map_data = gpd.read_file(filename = filename,                                   
                                   index_col = index_col)
    # rename fips code to match variable name in COVID-19 data
    map_data.rename(columns={"State":"state"},
                    inplace = True)
    # Combine statefips and county fips to create a single fips value
    # that identifies each particular county without referencing the 
    # state separately
    map_data[FIPS_name] = map_data["STATEFP"].astype(str) + \
        map_data["COUNTYFP"].astype(str)
    map_data[FIPS_name] = map_data[FIPS_name].astype(np.int64)
    # set FIPS as index
    map_data.set_index(FIPS_name, inplace=True)
    
    return map_data


In [7]:
map_data = import_geo_data(filename = "countiesWithStatesAndPopulation.shp",
    index_col = "date", FIPS_name= "fips_code")
counties = covid_data.index
covid_map_data= map_data.loc[counties]
for key, val in covid_data.items():
    covid_map_data[key] = val

In [16]:
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import *
from ipywidgets.embed import embed_minimal_html
from mpl_toolkits.axes_grid.inset_locator import inset_axes
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import cm
import matplotlib.ticker as mtick

plt.rcParams["font.size"] = 30
class geoPlot():
    def __init__(self, map_data, panel_data, state_data):
        self.interact_map_data = map_data.fillna(0)
        self.state_data = state_data
        self.panel_data = panel_data
        self.keys = ["Deaths per Million", "Cases per Million", 
                     "Daily Cases per Million MA", "Daily Deaths per Million MA"]
        self.dates = self.panel_data.groupby("date").mean().index
        self.states = list(state_dict.keys())
        self.states.insert(0, "United States")
        self.vmax = {key: self.panel_data[key].max() for key in self.keys}

        
    def geo_plot(self, date, column, state):
        def setup_cbar():
            cmap = cm.get_cmap('Reds', 10)
            norm = cm.colors.Normalize(vmin = 0, vmax = self.vmax[column])


            size = "5%" 
            sm = cm.ScalarMappable(cmap=cmap, norm=norm)
            # empty array for the data range
            sm._A = []
            # make space for colorbar
            divider = make_axes_locatable(self.ax[1])
            cax = divider.append_axes("right", size = size, pad = 0.1)
            # add colorbar to figure
            cbar = self.fig.colorbar(sm, cax=cax, cmap = cmap)
            cbar.ax.tick_params(labelsize=18)
            vals = list(cbar.ax.get_yticks())
            vals.append(self.vmax[column])
            # format display of values on colorbar
            cbar.ax.yaxis.set_major_formatter(mtick.Formatter())
            # format colorbar values as int
            cbar.ax.set_yticklabels([int(x) for x in vals])
            cbar.ax.set_ylabel(column, fontsize = 20)
            
        self.fig, self.ax = plt.subplots(2,1, figsize=(16,16))
        setup_cbar()
        self.state_data[column].plot(ax = self.ax[0], legend = False,
                                    color = "k", alpha = .3, linewidth = .5)
        self.ax[0].set_title(str(date)[:10]+ "\n" + column , fontsize = 36)
        self.date_line = self.ax[0].axvline(date, linewidth = 3, color = "C0", ls = "--")   
    
        if state == "United States":
            self.state_data[column]["United States"].plot(ax = self.ax[0], legend = False,
                                    color = "C3", alpha = 1, linewidth = 5)
            self.interact_map_data[~self.interact_map_data["state"].str.contains(
                "Alaska|Hawaii")].plot(column =  (column, date),
                                                 cmap = "Reds", ax=self.ax[1],
                                                 vmin = 0, vmax = self.vmax[column])
            self.axins = {}
            self.axins["Alaska"] = inset_axes(self.ax[1], width="17%", height="35%", loc="lower left")
            self.axins["Hawaii"] = inset_axes(self.ax[1], width="50%", height="40%", loc="lower left")
            for st in self.axins.keys():
                self.axins[st].set_xticks([])
                self.axins[st].set_yticks([])
                self.axins[st].axis("off")
                self.interact_map_data[self.interact_map_data["state"].str.contains(st)].plot(
                    column = (column,date),  ax = self.axins[st], cmap = "Reds", 
                    vmin = 0, vmax = self.vmax[column])
            
            self.axins["Hawaii"].set_xlim(-161, -155)
            self.axins["Alaska"].set_ylim(53, 71)
        else:
            self.interact_map_data[self.interact_map_data[
                "state"] ==state].plot(column =  (column,date), 
                                                 cmap = "Reds", ax=self.ax[1],
                                                 vmin = 0, vmax = self.vmax[column])
            self.state_data[column, state].plot(ax = self.ax[0], legend = False,
                                    color = "C3", alpha = 1, linewidth = 5)
        self.ax[0].set_xlabel("")
        self.ax[1].set_title(state , fontsize = 36)
interactive_map = geoPlot(covid_map_data, covid_panel, state_df_pivot)
interact(interactive_map.geo_plot, date = interactive_map.dates, 
            column = interactive_map.keys, 
            state = interactive_map.states)


interactive(children=(Dropdown(description='date', options=(Timestamp('2020-01-22 00:00:00'), Timestamp('2020-…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>